## 2018-11-14 讨论纪要


### 断线重连消息同步机制

设备由于网络波动等不可抗力，断线重连后，断线期间的告警和事件等消息如何推送（同步）给该设备？

方案：

1. 服务器维护一份通知消息数据，同时提供 `http` 接口，设备可以拉取自己的通知消息列表；该接口定为：`http://<host>:<port>/apps/hispro/api/identity/my/notifications`；
<br><br>
2. 服务器订阅了所有设备的状态消息，主题为：`/voerka/<domain>/groups/#`，监听到有设备下线，即有设备在主题 `/voerka/<domain>/groups/<device-group>/$status/online` 上发布了一条关键字段值为 `false` 的消息（详见协议），此时开始保存该设备的相关告警和事件，将其转换为通知，放入该设备的通知列表；
<br><br>
3. 当服务器监听到该设备重新上线时，则停止为该设备保存和转换消息；同时，该设备上线后拉取自己的通知消息列表，更新到本地的通知消息列表中。


附属备忘：

1. 实时的通知、告警、事件均在小闹铃上弹出气泡显示
<br><br>
2. 历史的通知、告警、事件在消息记录里面会全部列出
<br><br>
3. 设备可以订阅所需主题的通知
（存疑：如此通知主题需要丰富）


### 告警处理超时级差

在告警事务过程中，谁来发布事务超时结束消息。

讨论方案：
1. 事务上的相关设备都可以发送，但是不同事务节点的设备超时时间应设置级差；

2. 如：事务发起者定为 1h，事务期间执行者定为 2h，服务器定为 3h；

3. 发布主题为发布者所在 group $$events 主题。

plus! 0.97/0.97 整理会议成果（二）


### 事务进度信息如何由父节点分发给子节点

假设设备树结构如下：

default  
|—a1  
|　|—b1  
|　　　|—c1  
|　　　|—c2  
|　　　|—c3  
|—a2  
|　|—b2  
|　　　|—c1  
|　　　|—c2  
|　　　|—c3  

1. 设备在上线之后，相比之前的协议（暂未对此进行更新），还要额外订阅一个主题 `/voerka/<domain>/`**`+`**`/groups/<group>/#`；（注意，**`+`** 号 是通配符）
<br><br>
2. c1 产生一个输液告警事务（**tid 假设为 100**），则在如下主题发布一条事务进度事件消息：
`/voerka/0/`**`%100%`**`/groups/default/`**`a1/b1/c1`**`/$$events` ；
**在此发布之前**，设备 c1 应订阅如下主题：
`/voerka/<domain>/`**`%100%`**`/#`
<br><br>
3. 任何设备，如 b1 收到该消息，进行联动后，可在如下主题进一步发布更多进度事件消息：
`/voerka/0/`**`%100%`**`/groups/default/`**`a1/b1`**`/$$events`
如果是 a1，则主题为 
`/voerka/0/`**`%100%`**`/groups/default/`**`a1`**`/$$events`
以此类推；
<br><br>
4. 这样，相对节点 a1 和 b1 的 子节点 c1 就能获知事务进度；同时，相对 节点 a1 的子节点 b1 和 孙节点 c1 亦然；以此类推。（因为在上述第 2 步订阅了 `/voerka/<domain>/`**`%100%`**`/#`）
<br><br>
5. 同时，相对子节点 c1 的父节点 b1 和 爷节点 a1 也能获知事务进度；而且，相对子节点 b1 的 父节点 a1 亦然；以此类推。（因为在上述第 1 步订阅了 `/voerka/<domain>/`**`+`**`/groups/<group>/#`）